<a href="https://colab.research.google.com/github/tsakailab/prml/blob/master/ipynb/ex_MNIST_PCA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PCAによる低次元化（dimensionality reduction by principal component analysis）

主成分分析（PCA）でデータを低次元空間に埋め込む圧縮，再構成による近似の性質を観察しましょう．

参考資料：
- [Principal component analysis@Wikipedia](https://en.wikipedia.org/wiki/Principal_component_analysis)
- [Principal component analsys@scikit-learn](https://scikit-learn.org/stable/modules/decomposition.html#pca)

----

氏名：

学生番号：

----
## 基本課題（必須）

    1. Fashion-MNISTの 0，2，4，6 の 4 クラスからなる画像集合を主成分分析で低次元化したとき，
       上位の主成分はどのような画像の違いを表現していますか．

（ここに回答を書いてください）



    2. Fashion-MNISTの 0，2，4，6 の 4 クラスからなる画像集合を上位の主成分を用いて圧縮・再構成したとき，
       画像のどのような特徴が主に再構成され易い・され難いですか．また，その原因を考えてください．

（ここに回答を書いてください）



    3. 再構成誤差は，主成分の数に対してどのように依存しますか．特に，主成分の数が小さ過ぎる・大き過ぎるとき，
       どのような原因で訓練画像の再構成誤差が大きく・小さくなると考えられますか．
       ノイズの大小や訓練データ数による違いも踏まえて考察してください．

（ここに回答を書いてください）



    4.その他，気づいたこと，調べたことを書いてください．

（ここに回答を書いてください）


----
発展課題（任意）がこのノートブックの後半にあります．

### データ集合を取得します．

In [ ]:
from torchvision import datasets

# [MNIST](http://yann.lecun.com/exdb/mnist/)
#tvds = datasets.MNIST('./data', download=True, train=True)

# [Fashion-MNIST](https://github.com/zalandoresearch/fashion-mnist)
tvds = datasets.FashionMNIST('./data', download=True, train=True)

# [Kuzushiji-MNIST](https://github.com/rois-codh/kmnist)
#tvds = datasets.KMNIST('./data', download=True, train=True)

Ximages_all, y_all = tvds.data.numpy(), tvds.targets.numpy()

# [EMNIST](https://pytorch.org/vision/stable/generated/torchvision.datasets.EMNIST.html)
#tvds = datasets.EMNIST('./data', download=True, train=True, split='letters')
#Ximages_all, y_all = tvds.data.transpose_(-1,-2).numpy(), tvds.targets.numpy() - 1

In [ ]:
#@title ☆画像数とサイズを確認します（画像を加工したい場合はこのセルを編集して利用してください）．
import numpy as np
from skimage.filters import gaussian
from skimage.exposure import equalize_hist as eh
from skimage.transform import resize

Ximages = Ximages_all.copy()
y = y_all.copy()

'''
* blurring (https://scikit-image.org/docs/stable/api/skimage.filters.html#skimage.filters.gaussian)
* histogram equalization (https://scikit-image.org/docs/stable/auto_examples/color_exposure/plot_equalize.html)
* resize images (https://scikit-image.org/docs/stable/auto_examples/transform/plot_rescale.html)
'''
#Ximages = gaussian(np.float32(Ximages_all.transpose((1,2,0))), sigma=1.0, multichannel=True).transpose(2,0,1)
#Ximages = eh(Ximages_all.transpose((1,2,0))).transpose(2,0,1) * 255
#height, width = 8, 8; Ximages = resize(np.float32(Ximages_all.transpose((1,2,0))), (height, width)).transpose(2,0,1)

''' simulate noisy images '''
#Ximages = Ximages + np.random.rand(*Ximages.shape) * 200; Ximages = np.clip(Ximages,a_min=0,a_max=255)

Ximages = np.uint8(Ximages)
print("(#images, height, width)", Ximages.shape)
print(Ximages.dtype, ", max. pixel value = ", Ximages.max())

MNIST class labels

| [MNIST](http://yann.lecun.com/exdb/mnist/) | 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 |
| - | :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: |
| [Fashion-MNIST](https://github.com/zalandoresearch/fashion-mnist) | T-shirt/top | Trouser | Pullover | Dress | Coat | Sandal |  Shirt | Sneaker | Bag | Ankle boot |
| [Kuzushiji-MNIST](https://github.com/rois-codh/kmnist) | お | き | す | つ | な | は |  ま | や | れ | を |

In [ ]:
#@title 画像を表示する関数 `implot` を定義し，画像を例示します．
%matplotlib inline
!wget -q -N https://gist.githubusercontent.com/tsakai-g/c54a8fa059f9c655290586ec1cc2ec7b/raw/implot.py
%run implot.py

print("%d images in total" % len(y))
p = np.random.permutation(len(y))
implot(Ximages[p], y[p], num=16, vmin=0, vmax=255)

### クラスを抜粋したい場合（さもなくば実行しなくてよいです）
- `classes` でクラスを指定します．例：`classes = [1,4,7,9]`

In [ ]:
classes = [0,2,4,6]  # choose from 0 to 9

classes = np.unique(classes)
is_in_classes = list(map(lambda c: y == c, classes))
Ximages = Ximages[np.logical_or.reduce(is_in_classes)]
y = y[np.logical_or.reduce(is_in_classes)]

p = np.random.permutation(len(y))
implot(Ximages[p], y[p], num=20, vmin=0, vmax=255)

### ☆訓練データと検証データに分けます．
- 画像を `Ximages_train` と `Ximages_val` に分けます．それぞれ `n_train` 枚と `n_val` 枚の画像です．
- `Ximages_train` と `Ximages_val` を，それぞれ shape が `(n_train, 画素数)`，`(n_val，画素数)` の NumPy 配列にしたものが `X_train`，`X_val` です．

In [ ]:
from sklearn.model_selection import train_test_split
train_size = 0.8

# split the data into training and validation sets
Ximages_train, Ximages_val, y_train, y_val = train_test_split(Ximages, y, train_size=train_size)

n_train = len(Ximages_train)
n_val = len(Ximages_val)

# reshape to 28*28=784-dimensional feature vectors
X_train = np.reshape(Ximages_train, (Ximages_train.shape[0], -1))   # (n_train, 784)
X_val = np.reshape(Ximages_val, (Ximages_val.shape[0], -1))         # (n_val, 784)
print("X_train.shape = ", X_train.shape)
print("X_val.shape = ", X_val.shape)

## ★訓練データに対して主成分分析を実行します．
- [sklearn.preprocessing.StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html)を使用して，標準化の前処理を施します．`X_train` を標準化したものが `Xs_train` です．この標準化に従って `X_val` を標準化したものが `Xs_val` です．
- [sklearn.decomposition.PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html)を使用します．引数（特に`n_components`）の仕様とデフォルトの値を確認してください．
- `model.fit` は，標本平均 $\hat{\boldsymbol{\mu}}$ と，$k=$ `n_components` 本の主成分（分散共分散行列の固有ベクトル）$\boldsymbol{U}_{:k}$ を計算します．
- クラスラベル `y_train` と `y_val` は主成分分析に必要ありません．

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
Xs_train = scaler.fit_transform(X_train)
Xs_val = scaler.transform(X_val)

n_components=100

from sklearn.decomposition import PCA
model = PCA(n_components=n_components)
model.fit(Xs_train)

### 標本平均 $\hat{\boldsymbol{\mu}}$ および主成分 $\boldsymbol{U}_{:k}$ を可視化します．
- 標本平均は `model.mean_` です．標準化から0～255の画素値に戻して表示します．
- 正を赤，負を青の画素値とする画像として各主成分を表示します．

In [ ]:
h, w = Ximages.shape[1],Ximages.shape[2]
implot(scaler.inverse_transform(model.mean_.reshape(1,-1)).reshape(-1,h,w), vmin=0, vmax=255)
implot(model.components_.reshape(-1,h,w), num=min(20,n_components))

### 主成分分析でデータを低次元化します．
- `Xs_train` と `Xs_val` を低次元化したものが `Z_train` と `Z_val` です．`model.transform` はデータ $\boldsymbol x$ から $k$ 次元ベクトル $\boldsymbol z$ を次式で計算します．
$\boldsymbol{z}=\boldsymbol{U}_{:k}^\top(\boldsymbol{x}-\hat{\boldsymbol{\mu}})$

In [ ]:
Z_train = model.transform(Xs_train)
Z_val = model.transform(Xs_val)

In [ ]:
#@title `Z_train` と `Z_val` の第1，2，3成分を，ラベル `y` で色分けしてプロットします．<p>3次元データ X を y で色分けして立方体内にプロットする関数 `plot_embedding3d(X, y)` を定義します．</p>
%matplotlib inline
!wget -q -N https://gist.githubusercontent.com/tsakai-g/a0730f3692dd915a19a8e7598806e1fe/raw/plot_embedding3d.py
%run plot_embedding3d.py

In [ ]:
# 全部表示すると多いので，ランダムに n 個だけ表示する．
nt = min(300, n_train)
pt = np.random.choice(X_train.shape[0], nt, replace=False)
plot_embedding3d(Z_train[pt], y_train[pt])

nv = min(300, n_val)
pv = np.random.choice(Z_val.shape[0], nv, replace=False)
plot_embedding3d(Z_val[pv], y_val[pv])

MNIST class labels

| [MNIST](http://yann.lecun.com/exdb/mnist/) | 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 |
| - | :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: |
| [Fashion-MNIST](https://github.com/zalandoresearch/fashion-mnist) | T-shirt/top | Trouser | Pullover | Dress | Coat | Sandal |  Shirt | Sneaker | Bag | Ankle boot |
| [Kuzushiji-MNIST](https://github.com/rois-codh/kmnist) | お | き | す | つ | な | は |  ま | や | れ | を |

## 主成分の数 $k$ とノイズに対する再構成の依存性を調べます．
- `X_train` と `X_val` にノイズを加えたデータを `Xn_train` ， `Xn_val` とします．
- `Xn_train` と `Xn_val` を標準化後，主成分分析で `Z_train` と `Z_val` に低次元化します．
- $\hat{\boldsymbol{x}}=\boldsymbol{U}_{:k}\boldsymbol{z}_{:k}+\hat{\boldsymbol{\mu}}$ を計算して返す関数 `inverse_transform` を定義します．`k=n_components` のとき，`inverse_transform(model, Z, n_components)` は [`model.inverse_transform(Z)`](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html#sklearn.decomposition.PCA.inverse_transform) と同じです．
- `k` を指定して再構成し，ノイズを加えていない元画像と再構成画像を比較します．

In [ ]:
# add noise
noise_std = 50 # [0,255]
Xn_train = X_train + np.random.randn(*X_train.shape) * noise_std
Xn_val = X_val + np.random.randn(*X_val.shape) * noise_std

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
Xs_train = scaler.fit_transform(Xn_train)
Xs_val = scaler.transform(Xn_val)

n_components=300
from sklearn.decomposition import PCA
model = PCA(n_components=n_components, svd_solver='full').fit(Xs_train)

Z_train = model.transform(Xs_train)
Z_val = model.transform(Xs_val)

In [ ]:
def inverse_transform(model, Z, k):
    UkT = model.components_[:k]
    mu = model.mean_
    Xr = Z[:,:k].dot(UkT) + mu
    return Xr

def show_reconst(model, Z, k, X, scaler, num=20):
    Xr = inverse_transform(model, Z, k)
    Xr = scaler.inverse_transform(Xr)
    implot(Xr.reshape(-1,h,w), cmap=plt.cm.gray, vmin=0, vmax=255, num=num)
    implot((Xr - X).reshape(-1,h,w), num=num)

In [ ]:
k = 5   # must be no greater than n_components

num = 20
print("Train: original (1st row), original with noise (2nd row), reconstructed (3rd row), and error [-255(blue),255(red)] (4th row).")
implot(X_train.reshape(-1,h,w), cmap=plt.cm.gray, vmin=0, vmax=255, num=num)
implot(Xn_train.reshape(-1,h,w), cmap=plt.cm.gray, vmin=0, vmax=255, num=num)
show_reconst(model, Z_train, k, X_train, scaler, num=num)

plt.show();print("\n")

print("Val: original (1st row), original with noise (2nd row), reconstructed (3rd row), and error [-255(blue),255(red)] (4th row).")
implot(X_val.reshape(-1,h,w), cmap=plt.cm.gray, vmin=0, vmax=255, num=num)
implot(Xn_val.reshape(-1,h,w), cmap=plt.cm.gray, vmin=0, vmax=255, num=num)
show_reconst(model, Z_val, k, X_val, scaler, num=num)

### $k$ に対する再構成誤差を調べます．

In [ ]:
def compute_error(model, Z, k, X, scaler):
    Xr = inverse_transform(model, Z, k)
    Xr = scaler.inverse_transform(Xr)
    Err = Xr - X
    mse = np.sqrt(np.mean(Err**2))
    std = np.std(Err)
    return mse, std

k_components = np.arange(5, n_components, 10)
MSE_train, MSE_val = [], []
for k in k_components:
    mse, _ = compute_error(model, Z_train, k, X_train, scaler)
    MSE_train.append(mse)
    mse, _ = compute_error(model, Z_val, k, X_val, scaler)
    MSE_val.append(mse)

plt.plot(k_components, MSE_train, '.', label="train")
plt.plot(k_components, MSE_val, '+', label="val")
plt.ylim([0,None])
plt.legend()

--------
## 発展課題（任意）

    1. PCAを実装してください．

（回答は「★PCAの実装」に書いてください）

    2. 特異値分解（singular value decomposition; SVD）について調査し，
       標本分散共分散行列の固有ベクトルが SVD で計算できることを説明してください．

（ここに回答を書いてください）

### ★PCAの実装

`myPCA`という名のクラスとして実装しましょう．
[sklearn.decomposition.PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) に似た仕様とします．

訓練データ `X_train` から主成分を計算したあと，`X_val` を低次元化するには，
```
    model = myPCA(n_estimators=300)
    model.fit(X_train)
    Z_val = model.transform(X_val)
```
のように使うことを想定します．
- 訓練データの次元数`d`，個数`n`の場合，`X_train` は shape が `(n,d)` のNumPy配列です．
- `myPCA` の引数 `n_components` は主成分（固有ベクトル）の数です．

In [ ]:
import numpy as np
from scipy.linalg import svd

class myPCA:

    def __init__(self, n_components):
        self.n_components = n_components
        self.components_ = None
        self.mean_ = None
        self.singular_values_ = None

    def fit(self, X):                                       # X(n,d)
        # X の標本平均を計算します．
        self.mean_ = ''' np.mean() を使う '''

        # 標本分散共分散行列の固有ベクトルを得ます．
        _, sv, ev = svd(X - self.mean_, full_matrices=False)
        self.components_ = ev[0:self.n_components]
        self.singular_values_ = sv[0:self.n_components]     # (n_components,d)
        return self

    def transform(self, X):
        if self.mean_ is not None:
            X_transformed = ''' self.mean_ と self.components を用いて X を低次元化します '''
        return X_transformed

お疲れさまでした．

In [ ]:
#@title 付録：[MedMNIST](https://github.com/MedMNIST/MedMNIST)を使う場合<p>以下を実行後，このJupyter Notebook前半の「☆画像数とサイズを確認します（画像を加工したい場合はこのセルを編集して利用してください）．」以降を実行できます．
!pip install -q medmnist
import medmnist
print(f"MedMNIST v{medmnist.__version__} @ {medmnist.HOMEPAGE}")

In [ ]:
# 'pneumoniamnist', 'chestmnist', 'octmnist', 'breastmnist', 'tissuemnist', 'organamnist', 'organcmnist', 'organsmnist'
data_flag = 'pneumoniamnist'
DataClass = getattr(medmnist, medmnist.INFO[data_flag]['python_class'])

tvds = DataClass(split='train', download=True)
#tvds = DataClass(split='test', download=True)

print(tvds)
Ximages_all, y_all = tvds.imgs, tvds.labels.ravel()